In [1]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError
# import MySQLdb 
# check = False
# import json

In [2]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [3]:
sql = MySQLConnection('oskars_s','ec2-13-53-140-41.eu-north-1.compute.amazonaws.com','classicmodels')

Enter password for user oskars_s········


In [5]:
sql.query("show tables")

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,order_cust
4,orderdetails
5,orderdetails_income
6,orderdetails_incomes
7,orderdetails_w_income
8,orders
9,payments


In [ ]:
# sql.query("create table orderdetails_w_income as select *, priceeach*quantityordered as income from orderdetails")

In [6]:
ienakumi = """
select 
    sum(income) as kopejie_ienakumi,
    city
from 
    customers join 
    (
    select
        orders.customernumber,
        orderdetails_w_income.income
    from 
        orderdetails_w_income join orders on
        orderdetails_w_income.ordernumber = orders.ordernumber
    ) pasutijumu_ienakumi on 
    customers.customernumber = pasutijumu_ienakumi.customernumber
group by
    city
order by 
    kopejie_ienakumi desc
"""

sql.query(ienakumi)

,kopejie_ienakumi,city
0,979880.77,Madrid
1,591827.34,San Rafael
2,497941.50,NYC
3,386514.63,Auckland
4,263997.78,Singapore
...,...,...
72,42570.37,Brisbane
73,41506.19,Los Angeles
74,31310.09,Munich
75,29217.18,Charleroi


In [7]:
sql.query("SELECT status, count(*) as statusi from orders group by status")

,status,statusi
0,Cancelled,6
1,Disputed,3
2,In Process,6
3,On Hold,4
4,Resolved,4
5,Shipped,303


In [13]:
# Uzdevums:
# Sastādiet klientu sarakstu ar kuriem būtu jāsazinās, lai atjaunotu kādu iztrūkstošu personīgo informāciju
sql.query("select * from customers order by rand() limit 20")

manual_q = """
select * 
from 
    customers 
where 
    -- customernumber is NULL or
    -- customerName is NULL or
    contactlastname is NULL or
    contactfirstname is NULL or
    phone is NULL or
    addressline1 is NULL or
    -- addressline2 is NULL or
    city is NULL or
    state is NULL or
    postalcode is NULL
"""
sql.query(manual_q)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0
2,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0
3,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,None,Warszawa,None,01-012,Poland,NaN,0.0
4,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,None,Frankfurt,None,60528,Germany,1504.0,59700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,480,"Kremlin Collectables, Co.",Semenov,Alexander,+7 812 293 0521,2 Pobedy Square,None,Saint Petersburg,None,196143,Russia,NaN,0.0
70,481,"Raanan Stores, Inc","Altagar,G M",Raanan,+ 972 9 959 8555,3 Hagalim Blv.,None,Herzlia,None,47625,Israel,NaN,0.0
71,484,"Iberia Gift Imports, Corp.",Roel,José Pedro,(95) 555 82 82,"C/ Romero, 33",None,Sevilla,None,41101,Spain,1702.0,65700.0
72,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300.0


In [ ]:
# Uzdevums: 
# 1. Atlasiet kopējos ienākumus no visiem pasūtījumiem, 
# 2. Atlasiet kopējos ienākumus tikai no pasūtījumiem, kas ir piegādāti (shipped)

In [51]:
# Uzdevums:
# 1. Atlasiet 10 visvairāk pārdotās preces

sold = """
select
    productcode,
    sum(quantityordered) kopskaits
from
    orderdetails
group by 
    productcode
order by 
    kopskaits desc
limit 
    10
"""

sold_2 = """
SELECT
    productName as produkts,
    sum(quantityOrdered) as daudzums
FROM
    products join orderdetails on
    products.productCode = orderdetails.productCode
GROUP BY
    produkts
ORDER BY
    daudzums desc
LIMIT 10
"""
# sql.query(sold)


# 2. Izpētiet datus un izdomājiet pie kādiem nosacījumiem preču skaits būtu jāpapildina 
# (pieņemsim, ka pārdošanai pieejamo preču skaits ir papildināms par jebkādu skaitu 2 dienās)

# No sākuma mēģinu selektēt tabulu, kura satur pirktos apjomus katrā dienā katrai precei
# Cik katrā dienā katra prece tiek pasūtīta?
q1 = """
select 
    productcode,
    avg(soldperday) as avg_sold
from (
    select 
        productcode,
        orderdate,
        sum(quantityordered) as soldperday
    from 
        orders join orderdetails on
        orders.ordernumber = orderdetails.ordernumber
    group by 
        productcode, orderdate
    ) smthn
group by 
    productcode
"""
sql.query(q1)



# 3. Atlasiet preces, kuras būtu jāiepērk pēc šiem izvēlētajiem kritērijiem

,productcode,avg_sold
0,S10_1678,37.7500
1,S10_1949,34.3214
2,S10_2016,35.6786
3,S10_4698,35.1786
4,S10_4757,36.7857
...,...,...
104,S700_3505,35.2593
105,S700_3962,33.1852
106,S700_4002,38.7500
107,S72_1253,34.2857


In [43]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš saskaita cik ir darbinieku, kuriem ir padotie
q = """
select distinct *
select 
    firstname, 
    lastname, 
    employeenumber
from 
    employees right join 
    (
    select
        distinct reportsto,
        employeenumber as enum
    from 
        employees
    ) bosses on bosses.reportsto = employees.employeenumber
"""
sql.query(q)

,firstname,lastname,employeenumber,enum
0,None,None,NaN,1002
1,Diane,Murphy,1002.0,1056
2,Diane,Murphy,1002.0,1076
3,Mary,Patterson,1056.0,1088
4,Mary,Patterson,1056.0,1102
5,Mary,Patterson,1056.0,1143
6,Mary,Patterson,1056.0,1621
7,William,Patterson,1088.0,1611
8,William,Patterson,1088.0,1612
9,William,Patterson,1088.0,1619


In [37]:
q2 = """select
        distinct reportsto,
        employeenumber as enum
    from 
        employees"""
sql.query(q2)

,reportsto,enum
0,NaN,1002
1,1002.0,1056
2,1002.0,1076
3,1056.0,1088
4,1056.0,1102
5,1056.0,1143
6,1056.0,1621
7,1088.0,1611
8,1088.0,1612
9,1088.0,1619


In [13]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš katram customer id piekārto visnesenāko maksājumu
recent = """
select 
    customernumber,
    max(paymentdate) as 'jaunakais maksajums'
from 
    payments
group by
    customernumber

"""
display(sql.query("show tables"))
display(sql.query(recent))

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,order_cust
4,orderdetails
5,orderdetails_income
6,orderdetails_incomes
7,orderdetails_w_income
8,orders
9,payments


,customernumber,jaunakais maksajums
0,103,2004-12-18
1,112,2004-12-17
2,114,2004-12-15
3,119,2005-02-22
4,121,2004-11-28
...,...,...
93,486,2004-11-23
94,487,2004-02-29
95,489,2004-01-31
96,495,2004-05-14


In [32]:
# Uzdevums: 
# Uzrakstiet python funkciju, kuras input ir valīds produkta kods un output ir json teksta pavediens, 
# kas satur lauka nosaukumu un informāciju par preci.
# Piem. ja funkcijas input būtu S700_3505, 
# tad funkcijai būtu jāatgriež sekojošs teksta pavediens (piemērā sadalīts vairākās rindiņās lasāmībai):



# {
# "product_description":"Completed model measures 19 1\\/2 inches long, 9 inches high, 3inches 
# wide and is in barn red\\/black. All wood and metal.",
# "product_name":"The Titanic",
# "original_price": 51.09,
# "quantity_in_stock":1956
# }

import json

def produkts(kods):
    # # sql.query("select productdescription from products where productcode = 'S700_3505'")
    vaic1 = """
    select 
        productdescription as product_description, 
        productname as product_name, 
        buyprice as original_price, 
        quantityinstock as quantity_in_stock 
    from 
        products 
    where 
        productcode = '"""
    vaic2="'"
    df = sql.query(vaic1+kods+vaic2)
    display(df)
#     print(df.to_json(orient='records'))
    # Alternatīva, iekļaujot arī tukšu ierakstu ķeršanu:
    if not df.empty:
        df_dct = df.to_dict()
        for key in df_dct.keys():
            df_dct[key] = df_dct[key][0]
        df_str = json.dumps(df_dct)
    else:
        case_empty_dct = {'product_description':'default', 'product_name':'default', 'buyprice':0.0, 'quantityinstock':0} # Kas šeit iet iekšā?
        df_str = json.dumps(case_empty_dct)
    return df_str

produkts("S700_3505")

,product_description,product_name,original_price,quantity_in_stock
0,"Completed model measures 19 1/2 inches long, 9...",The Titanic,51.09,1956


'{"product_description": "Completed model measures 19 1/2 inches long, 9 inches high, 3inches wide and is in barn red/black. All wood and metal.", "product_name": "The Titanic", "original_price": 51.09, "quantity_in_stock": 1956}'

In [ ]:
# Uzdevums:
# Papildiniet šo funkciju ar funkcionalitāti, kas json pavedienā, string mainīgo vietā atgriež "default" un skaitļu vietā 0, 
# ja nav atrasts neviens produkts ar šādu produkta kodu